In [1]:
# for use in tutorial and development; do not include this `sys.path` change in production:
import sys ; sys.path.insert(0, "../")
import os ; from os.path import dirname

In [2]:
import kglab

namespaces = {
    "foaf": "http://xmlns.com/foaf/0.1/",
    "gorm": "http://example.org/sagas#",
    "rel":  "http://purl.org/vocab/relationship/",
    }

kg = kglab.KnowledgeGraph(
    name = "Happy Vikings KG example for SKOS/OWL inference",
    namespaces=namespaces,
    )

kg.load_rdf(dirname(os.getcwd()) + "/dat/gorm.ttl")

In [3]:
text = kg.save_rdf_text()
print(text)

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix gorm: <http://example.org/sagas#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

gorm:Astrid a gorm:Viking ;
    gorm:childOf gorm:Bodil,
        gorm:Leif ;
    foaf:topic_interest gorm:Fighting .

gorm:childOf rdfs:domain gorm:Viking ;
    rdfs:range gorm:Viking ;
    owl:inverseOf gorm:ancestorOf .

gorm:spouseOf a owl:SymmetricProperty ;
    rdfs:domain gorm:Viking ;
    rdfs:range gorm:Viking .

gorm:Berserkr a foaf:Thing ;
    skos:broader gorm:Fighting .

gorm:Bjorn a gorm:Viking ;
    gorm:childOf gorm:Gorm ;
    foaf:topic_interest gorm:Pilaging .

gorm:Bodil a gorm:Viking ;
    gorm:spouseOf gorm:Leif .

gorm:Gorm a gorm:Viking ;
    foaf:topic_interest gorm:Berserkr .

gorm:Pilaging a foaf:Thing ;
    skos:broader gorm:Fighting .

gorm:Leif a gorm:Viking ;
    gorm:childOf gorm:Bjorn .

gorm:Fighting a foaf:Th

In [4]:
sparql = """
SELECT ?child ?viking
  WHERE {
      ?child gorm:childOf ?viking
  }
  ORDER BY ASC(?viking)
  """

df = kg.query_as_df(sparql)
df

,child,viking
0,gorm:Leif,gorm:Bjorn
1,gorm:Astrid,gorm:Bodil
2,gorm:Bjorn,gorm:Gorm
3,gorm:Astrid,gorm:Leif


In [5]:
import networkx as nx

# IFF cuGraph
#import cugraph as nx

subgraph = kglab.SubgraphMatrix(kg, sparql, src_dst=["child", "viking"])
nx_graph = subgraph.build_nx_graph(nx.DiGraph(), bipartite=True)

In [6]:
raw_dog = nx.pagerank(nx_graph)
raw_dog

{0: 0.15938145155157013,
 1: 0.247320578151483,
 2: 0.11184682438205981,
 3: 0.15938145155157013,
 4: 0.322069694363317}

In [7]:
df = subgraph.build_df()
df

,src,dst
0,0,1
1,2,3
2,1,4
3,2,0


In [8]:
import networkx as nx

# IFF cuGraph
#df = df.to_pandas()

G = nx.from_pandas_edgelist(
        df,
        source="src",
        target="dst",
        create_using=nx.DiGraph(),
    )

type(G)

networkx.classes.digraph.DiGraph

In [9]:
results = nx.pagerank(G)
results

{0: 0.15938145155157013,
 1: 0.247320578151483,
 2: 0.11184682438205981,
 3: 0.15938145155157013,
 4: 0.322069694363317}

In [10]:
v0 = raw_dog.values()

# IFF cuGraph
#v0 = raw_dog.to_pandas().to_dict()["pagerank"].values()

v0 = sorted(list(map(lambda x: round(x, 4), v0)))
v0

[0.1118, 0.1594, 0.1594, 0.2473, 0.3221]

In [11]:
v1 = sorted(list(map(lambda x: round(x, 4), results.values())))
v1

[0.1118, 0.1594, 0.1594, 0.2473, 0.3221]

In [12]:
assert v0 == v1